# ECE/CS 434 | MP4: IMU PDR
<br />
<nav>
    <span class="alert alert-block alert-warning">Due at 11:59PM April 13th 2021 on Gradescope</span> |
    <a href="https://www.gradescope.com/courses/223105">Gradescope</a> | 
    <a href="https://courses.grainger.illinois.edu/cs434/sp2021/">Course Website</a> | 
    <a href="http://piazza.com/illinois/spring2021/csece434">Piazza</a>
</nav><br> 

**Name(s):** _ , _<br>
**NetID(s):** _ , _

<hr />  

## Objective
In this MP, you will:
- Implement a step counting algorithm using accelerometer data.
- Apply signal processing and linear algebra functions such as low/high pass filtering, peak detection, fft, etc. to the step counting algorithm.
- Calculate phone orientation using a single static accelerometer reading. 
- Track phone orientation through a sequence of gyroscope data by performing integration.

---
## Problem Overview
In pedestrian dead-reckoning applications, two pieces of information need to be tracked: how far a user walked, and the direction of the walk. In the first part of this MP, you will write a step counter using accelerometer data as input. In the second part, you will derive the initial orientation of the phone using a single accelerometer reading and calculate the final orientation using a sequence of gyroscope data.

---
## Imports & Setup

### Installing requirements correctly

First. we will make sure that the correct versions of required modules are installed. This ensures that your local Python environment is consistent with the one running on the Gradescope autograder. Just convert the following cell to code and run:

<div class="alert alert-block alert-info"><b>Note:</b> It's preferred that your local environment matches the autograder to prevent possible inconsistencies. However, if you're running into annoying Python version issues but haven't had any issues getting consistent results on the autograder, there is no need to stress over it. Just skip for now and come back when you do encounter inconsistencies:) Ditto below.
</div>

<div class="alert alert-block alert-info"><b>WARNING:</b> ENSURE THE FOLLOWING CELL IS MARKDOWN OR DELETED BEFORE SUBMITTING. THE AUTOGRADER WILL FAIL 
</div>

if __name__ == '__main__':
    import sys
    !{sys.executable} -m pip install -r requirements.txt

### Your imports
Write your import statements below. If Gradescope reports an error and you believe it is due to an unsupported import, check with the TA to see if it could be added.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal as signal
from scipy.spatial.transform import Rotation

# This function is used to format test results. You don't need to touch it.
def display_table(data):
    from IPython.display import HTML, display

    html = "<table>"
    for row in data:
        html += "<tr>"
        for field in row:
            html += "<td><h4>{}</h4><td>".format(field)
        html += "</tr>"
    html += "</table>"
    display(HTML(html))

### Sanity-check

Running the following code block verifies that the correct module versions are indeed being used. 

Try restarting the Python kernel (or Jupyter) if there is a mismatch even after intalling the correct version. This might happen because Python's `import` statement does not reload already-loaded modules even if they are updated.

In [2]:
if __name__ == '__main__':
    from IPython.display import display, HTML

    def printc(text, color):
        display(HTML("<text style='color:{};weight:700;'>{}</text>".format(color, text)))

    _requirements = [r.split("==") for r in open(
        "requirements.txt", "r").read().split("\n")]

    import sys
    for (module, expected_version) in _requirements:
        try:
            if sys.modules[module].__version__ != expected_version:
                printc("[✕] {} version should to be {}, but {} is installed.".format(
                    module, expected_version, sys.modules[module].__version__), "#f44336")
            else:
                printc("[✓] {} version {} is correct.".format(
                    module, expected_version), "#4caf50")
        except:
            printc("[–] {} is not imported, skipping version check.".format(
                module), "#03a9f4")

---
## Part 1. Step Counter
We have provided you with smartphone accelerometer data collected under three circumstances
<ol type="A">
  <li>walking with phone in pant pocket</li>
  <li>walking with phone held in the hand statically as if the user is looking at it while walking</li>
  <li>walking with phone in hand and the hand swinging</li>
</ol>
For each file, there are three columns, representing the accelerometer readings in three local axes(unit: $m / s^{2}$). The accelerometer is sampled at 100Hz.

Implement your algorithm in the `count_steps(walk_accl_file)` function below. Do NOT change the function signature. You are, however, free to define and use helper functions. You are expected to use common signal processing and linear algebra functions (e.g., high/low pass filtering, convolution, cross correllation, peak detection, fft etc.) 

In [3]:
def get_walk_dataset(accl_file):
    data_csv = pd.read_csv(accl_file, header=None, names=['x', 'y', 'z'])
    data = data_csv.to_numpy()
    
#     x = np.arange(data.shape[0])
#     plt.figure(figsize=(16, 10), dpi=144)
#     for i in range(3):
#         plt.plot(x, data[:, i])
#     plt.show()
    return data

def moving_average(data, period, offset):
    padded_data = np.pad(data, (0, period), 'wrap')
    average_data = np.zeros(data.shape[0])
    for i in range(data.shape[0]):
        average_data[i] = np.mean(padded_data[i:i+period])
    return average_data + offset

def get_count(data):
    # Naive
    steps = 0
    series_len = data.shape[0]
    average_accel_arr = np.zeros((series_len))
    for i in range(series_len):
        ax, ay, az = data[i]
        average_accel = np.sqrt(ax**2 + ay**2 + az**2)
        average_accel_arr[i] = average_accel
    
    order = 5
    sampling_freq = 100
    pad = 50
    cutoff_freq = 3
    
    normalized_cutoff_freq = 2 * cutoff_freq / sampling_freq
    numerator_coeffs, denominator_coeffs = signal.butter(order, normalized_cutoff_freq)
    padded_average_accel_arr = np.pad(average_accel_arr, (pad, pad), 'wrap')
    filtered_signal = signal.lfilter(numerator_coeffs, denominator_coeffs, padded_average_accel_arr)
    filtered_signal = filtered_signal[pad: -pad]
#     print(filtered_signal.shape)
    
    offset = 0
    moving_window = 200
    mean_filtered_accel = moving_average(filtered_signal, moving_window, offset)
#     mean_filtered_accel = np.pad(mean_filtered_accel, (pad, pad), 'wrap')
    
    
    starting_above = False
    if filtered_signal[0] > mean_filtered_accel[0]:
        starting_above = True # Starting above average
    flag = False  
    for i in range(series_len):
        if starting_above:
            if filtered_signal[i] > mean_filtered_accel[i] and flag == False:
                flag = True
            elif filtered_signal[i] < mean_filtered_accel[i] and flag == True:
                steps += 1
                flag = False
        else:
            if filtered_signal[i] < mean_filtered_accel[i] and flag == False:
                flag = True
            elif filtered_signal[i] > mean_filtered_accel[i] and flag == True:
                steps += 1
                flag = False
            
            
#     x = np.arange(series_len)
#     plt.figure(figsize=(16, 10), dpi=144)
#     plt.plot(x, mean_filtered_accel)
#     plt.plot(x, filtered_signal)
# #     plt.plot(x, average_accel_arr)
#     plt.show()
    return steps

In [4]:
# This function takes 1 argument:
#     walk_accl_file  (string) - name of data file for accelerometer data
# It returns an integer, the number of steps
def count_steps(walk_accl_file):
    # Your implementation starts here:
    walk_data = get_walk_dataset(walk_accl_file)
    steps = get_count(walk_data)
    return steps

### Run & Test
Use the cell below to run and test `count_steps(walk_accl_file)`. 

In [5]:
def estimate_steps_score(calculated, expected):
    delta = abs(calculated - expected)
    return 1 if(delta <= 2) else max((1 - abs(delta - 2) / expected), 0)


if __name__ == '__main__':
    walk_accl_files = ['data/holdstatic_20steps.csv', 'data/inpocket_26steps.csv',
                       'data/inpocket_36steps.csv', 'data/swing_32steps.csv', 'data/swing_38steps.csv']
    groundtruth = [20, 26, 36, 32, 38]
    output = [['Dataset', 'Expected Output', 'Your Output', 'Grade']]
    for i in range(len(groundtruth)):
        calculated = count_steps(walk_accl_files[i])
        score = estimate_steps_score(calculated, groundtruth[i])
        output.append([walk_accl_files[i], groundtruth[i],
                      calculated, "{:2.2f} / 5.00".format(score * 5)])
    output.append(['<i>👻 Hidden test 1 👻</i>','<i>???</i>', '<i>???</i>', '<i>???</i> / 15.00'])
    output.append(['<i>...</i>', '<i>...</i>', '<i>...</i>', '<i>...</i>'])
    output.append(['<i>👻 Hidden test 5 👻</i>','<i>???</i>', '<i>???</i>', '<i>???</i> / 15.00'])
    display_table(output)

---
## Part 2. Orientation Tracking

### Part 2.1 Initial Orientation Calculation
Assume the phone is static at the initial moment. We will provide you with the accelerometer reading at that moment (unit: $m / s^{2}$). Your goal is to identify the initial phone orientation from this reading. We will not provide compass data here since all the data are collected indoor and compass won’t give an accurate north indoor. Instead, assume at the initial moment, the projection of the phone’s local Y axis onto the horizontal plane is pointing towards the global Y axis. This will also give a fixed phone initial orientation.

**We expect you to output the global direction in which the phone’s local X axis is pointing at.**

<div class="alert alert-block alert-info"><b>Hint:</b> Find the global Y axis’s direction in the local frame and let this direction be a 3 × 1 vector $v_{1}$. Let the gravity in
the local frame be another 3 × 1 vector $v_{2}$. Then essentially you need to solve the following equation: <br> $
R\left[v_{1} v_{2}\right]=\left[\begin{array}{ll}
0 & 0 \\
1 & 0 \\
0 & 1
\end{array}\right]$ </div>

### Part 2.2 3D Orientation Tracking
In this part, you need to take the initial orientation calculated in part 1, and perform gyro integration for each timestamp onward. We will provide you with a trace of gyroscope data, in CSV format. There are three columns in the file, representing the gyroscope readings in three **local** axes (unit: $rad / s$). The gyroscope is sampled at 100Hz. Your task is to track the phone’s 3D orientation and **output the end direction in which the phone’s local X axis is pointing at in the global frame**.

One way of solving this problem can be:
<ol type="A">
    <li> Assume the gyroscope’s sample interval is $\Delta t$. </li>
    <li> Get the phone's instant rotation axis and rotation angle in the local frame $(\vec{l}, \Delta \theta)$ for each time stamp $t_{i},$ where $\vec{l}=\left(\omega_{x}, \omega_{v}, \omega_{z}\right)$ and $\Delta \theta=\sqrt{\left(\omega_{x}^{2}+\omega_{v}^{2}+\omega_{z}^{2}\right)} \cdot \Delta t$ </li>
    <li> Project the instant rotation axis $\vec{l}$ into the global frame using the phone's $3 \mathrm{D}$ orientation matrix $R_{i}$ at time $t_{i}$. </li>
    <li> Convert the instant rotation axis and angle in global frame into the form of rotation matrix $\Delta R_{i}$. </li>
    <li> Find the total 3D rotation matrix for time $t_{i+1}: R_{i+1}=\Delta R_{i} \cdot R_{i}$ </li>
</ol>

--- 
**Implement both algorithms in `track_orientation(orientation_accl_file, gyro_file)` below.** This is because the initial rotation matrix needed for calculating final orientation is a by-product of calculating initial orientation. Do NOT change the function signature. You are, however, free to define and use helper functions.

In [6]:
def get_accl_data(accl_file):
    data_csv = pd.read_csv(accl_file, header=None, names=['x', 'y', 'z'])
    data = data_csv.to_numpy()
    return data

def get_gyro_data(gyro_file):
    data_csv = pd.read_csv(gyro_file, header=None, names=['x', 'y', 'z'])
    data = data_csv.to_numpy()
    return data

def get_R0(accl_data):
    angles = []
    accl_data = np.array(accl_data)
    for i in range(accl_data.shape[0]):
        accl_x, accl_y, accl_z = accl_data[i][0], accl_data[i][1], accl_data[i][2]
        g = np.sqrt(accl_x**2 + accl_y**2 + accl_z**2)
        accl_x, accl_y, accl_z = accl_x/g, accl_y/g, accl_z/g
        v2 = np.array([accl_x, accl_y, accl_z])
        
        g_comp_y_on_z = accl_y**2
        accl_y_comp_z_vec = v2*g_comp_y_on_z
        accl_y_comp_y_vec = [0, accl_y, 0] - accl_y_comp_z_vec
        accl_y_comp_y_vec_norm = accl_y_comp_y_vec/np.linalg.norm(accl_y_comp_y_vec)
        v1 = accl_y_comp_y_vec_norm
        v0 = np.array([np.sqrt(1 - v1[i]**2 - v2[i]**2) for i in range(3)])
        v0 = v0/np.linalg.norm(v0)
        
        if v2[2] < 0: #local z in global z  
            v0[0] = -v0[0] # local x in global x
        if v2[0] > 0 : #local x in global z
            v0[2] = -v0[2] # local z in global x
        R = np.stack((v0, v1, v2))
        print(R)
        return R
    
def get_next_R(Ri, l_local, dt):
    dtheta= np.linalg.norm(l_local)*dt
    #step2
    l_global = Ri@l_local
    l_global = l_global/np.linalg.norm(l_global)
    #step3
    dRi = Rotation.from_rotvec(dtheta*l_global)
    dRi = dRi.as_matrix()
    R_temp = dRi@Ri
    R_next = np.ndarray(R_temp.shape)
    for i in range(3):
        R_next[i] = R_temp[i]/np.linalg.norm(R_temp[i])
    #step4
    return R_next

In [7]:
# This function takes 2 arguments:
#     - orientation_accl_file (string) - name of file containing a single accl reading
#     - gyro_file (string) - name of file containing a sequence of gyroscope data
# It returns two arguments: an array representing the initial global direction
# in which the phone's local X axis is pointing at, and the final.
def track_orientation(orientation_accl_file, gyro_file):
    # Your implementation starts here:
    
    accl_data = get_accl_data(orientation_accl_file)
    R0 = get_R0(accl_data)
    init_orientation = R0[:, 0]
    
    gyro_data = get_gyro_data(gyro_file)
    
    Hz=100
    dt = 1/Hz
    Ri = R0
    
    for i in range(gyro_data.shape[0]):
        Ri = get_next_R(Ri, gyro_data[i], dt)
    final_orientation = Ri[:, 0]
    
    return [
        init_orientation,
        final_orientation,
    ]  # [initial orientation], [final orientation]

### Run & Test
Use the cell below to run and test Part 2.

In [8]:
def get_deviation(calculated, expected):
    calculated = np.array(calculated)
    expected = np.array(expected)
    with np.errstate(divide='ignore', invalid='ignore'):
        dot_prod = np.dot(calculated, expected) / \
            np.linalg.norm(calculated) / np.linalg.norm(expected)
        return np.degrees(np.arccos(dot_prod))


if __name__ == '__main__':
    gt_init = [0.9999, -0.0020, 0.0120]
    gt_final = [-0.0353, 0.9993, 0.0076]
    stu_init, stu_final = track_orientation(
        'data/orientation_accl.csv', 'data/gyro.csv')

    output = [['Test', 'Dataset', 'Expected Output',
               'Your Output', 'Deviation', 'Result', 'Grade']]
    init_state = 'FAILED'
    final_state = 'FAILED'
    init_grade = 0
    final_grade = 0
    init_dev = get_deviation(stu_init, gt_init)
    final_dev = get_deviation(stu_final, gt_final)
    if(init_dev < 2):
        init_state = 'PASSED'
        init_grade = 10
    if(final_dev < 2):
        final_state = 'PASSED'
        final_grade = 10
    output.append(['Initial Orientation',
                  'orientation_accl.csv, gyro.csv', gt_init, stu_init, "{:2.2f}°".format(init_dev), init_state, "{} / 10".format(init_grade)])
    output.append(['Final Orientation', 'orientation_accl.csv, gyro.csv',
                  gt_final, stu_final, "{:2.2f}°".format(final_dev), final_state, "{} / 10".format(final_grade)])
    output.append(['<i>👻 Hidden test 1 👻</i>','<i>???</i>', '<i>???</i>', '<i>???</i>', '<i>???</i>', '<i>???</i>', '<i>???</i> / 10'])
    output.append(['<i>...</i>', '<i>...</i>', '<i>...</i>', '<i>...</i>', '<i>...</i>', '<i>...</i>', '<i>...</i>'])
    output.append(['<i>👻 Hidden test 4 👻</i>','<i>???</i>', '<i>???</i>', '<i>???</i>', '<i>???</i>', '<i>???</i>', '<i>???</i> / 10'])
    display_table(output)

[[ 9.99926462e-01  7.21399364e-09 -1.21272954e-02]
 [-2.04040710e-03  9.85744503e-01 -1.68236773e-01]
 [ 1.19544148e-02  1.68249146e-01  9.85672013e-01]]


---
## Rubric

#### Step Counting (100 points) 
You will be graded on the 5 sets of provided data (5 points each), as well as 5 sets of hidden data (15 points each). For each test case, the grade depends on how much the result deviates from the groudtruth. A 2-step error for the provided data is tolerated. A 4-step error for the hidden data is tolerated. For results greater than the error threshold, your score will be scaled proportionally.

####  Orientation Tracking (100 points) 
You will be graded on the provided data as well as 4 addition sets of data. They are each worth 20 points. A 2-degree error is tolerated. For results greater than the error threshold, no points will be rewarded since we provided a detailed algorithm to follow. The test data also include the simple case where the phone’s initial local frame is aligned with the global frame, and phone will only rotate along Z axis onwards. (In case you find the MP too difficult, only doing 1D integration on Z axis should at least give you some points.)

---
## Submission Guideline
This Jupyter notebook is the only file you need to submit on Gradescope. If you are working in a pair, make sure your partner is correctly added on Gradescope and that both of your names are filled in at the top of this file.

**Make sure any code you added to this notebook, except for import statements, is either in a function or guarded by `__main__`(which won't be run by the autograder). Gradescope will give you immediate feedback using the provided test cases. It is your responsibility to check the output before the deadline to ensure your submission runs with the autograder.**